In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Flatten
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alxmamaev/flowers-recognition")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flowers-recognition


In [5]:
def load_data(dataset_path):
    X, y = [],[]
    for cls in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, cls)
        if not os.path.isdir(class_path):
            continue
        images_paths =[os.path.join(class_path, img) for img in os.listdir(class_path)]
        labels= [cls]* len(images_paths)
        X.extend(images_paths)
        y.extend(labels)
    return X, y

X, y= load_data("/kaggle/input/flowers-recognition/flowers")

print("total samples:", len(X))
print("labels:", set(y))

total samples: 4317
labels: {'sunflower', 'rose', 'dandelion', 'daisy', 'tulip'}


In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

train_df = pd.DataFrame({"image path": x_train, "class": y_train})
test_df   = pd.DataFrame({"image path": x_test, "class": y_test})
train_df
x_train

['/kaggle/input/flowers-recognition/flowers/sunflower/3665455426_9cd1c3af4a_n.jpg',
 '/kaggle/input/flowers-recognition/flowers/dandelion/8497389500_45636fdd14.jpg',
 '/kaggle/input/flowers-recognition/flowers/dandelion/6250363717_17732e992e_n.jpg',
 '/kaggle/input/flowers-recognition/flowers/rose/8181940917_1ac63937d5_n.jpg',
 '/kaggle/input/flowers-recognition/flowers/daisy/4561871220_47f420ca59_m.jpg',
 '/kaggle/input/flowers-recognition/flowers/daisy/16323838000_3818bce5c6_n.jpg',
 '/kaggle/input/flowers-recognition/flowers/sunflower/15054864058_2edca122a9_n.jpg',
 '/kaggle/input/flowers-recognition/flowers/dandelion/459633569_5ddf6bc116_m.jpg',
 '/kaggle/input/flowers-recognition/flowers/daisy/8681746439_d6beeefbf9.jpg',
 '/kaggle/input/flowers-recognition/flowers/rose/2059172936_032ffc12aa.jpg',
 '/kaggle/input/flowers-recognition/flowers/tulip/4571993204_5b3efe0e78.jpg',
 '/kaggle/input/flowers-recognition/flowers/rose/15681454551_b6f73ce443_n.jpg',
 '/kaggle/input/flowers-recog

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)



In [8]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col="image path",
    y_col="class",
    target_size=(128, 128),
    class_mode="categorical",
    batch_size=32,
        subset="training",
    shuffle=True,
)

val_generator = val_datagen.flow_from_dataframe(
    test_df,
    x_col="image path",
    y_col="class",
    target_size=(128, 128),
    class_mode="categorical",
    batch_size=32,
    shuffle=False
)

Found 3453 validated image filenames belonging to 5 classes.
Found 864 validated image filenames belonging to 5 classes.


In [9]:
CNN_model = Sequential()
CNN_model.add(Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)))
CNN_model.add(MaxPooling2D(pool_size=(2,2)))
CNN_model.add(Conv2D(32, (3,3), activation='relu'))
CNN_model.add(Conv2D(32, (3,3), activation='relu'))

CNN_model.add(MaxPooling2D(pool_size=(2,2)))

CNN_model.add(Conv2D(64, (3,3), activation='relu'))

CNN_model.add(Conv2D(64, (3,3), activation='relu'))
CNN_model.add(MaxPooling2D(pool_size=(2,2)))
CNN_model.add(Dropout(0.25))

CNN_model.add(Conv2D(128, (3,3), activation='relu'))

CNN_model.add(Conv2D(128, (3,3), activation='relu'))
CNN_model.add(MaxPooling2D(pool_size=(2,2)))
CNN_model.add(Dropout(0.45))


CNN_model.add(Flatten())
CNN_model.add(Dense(128, activation='relu'))

CNN_model.add(Dense(64, activation='relu'))
CNN_model.add(Dense(5, activation='softmax'))
CNN_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
CNN_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 59, 59, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 29, 29, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 27, 27, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 25, 25, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 10, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 562,053 (2.14 MB)

 Trainable params: 562,053 (2.14 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = CNN_model.fit(train_generator, validation_data=val_generator,epochs=45)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/45
108/108 ━━━━━━━━━━━━━━━━━━━━ 38s 275ms/step - accuracy: 0.2333 - loss: 1.5826 - val_accuracy: 0.3414 - val_loss: 1.4212
Epoch 2/45
108/108 ━━━━━━━━━━━━━━━━━━━━ 23s 217ms/step - accuracy: 0.3808 - loss: 1.3568 - val_accuracy: 0.4421 - val_loss: 1.2103
Epoch 3/45
108/108 ━━━━━━━━━━━━━━━━━━━━ 23s 213ms/step - accuracy: 0.4766 - loss: 1.2119 - val_accuracy: 0.5231 - val_loss: 1.2307
Epoch 4/45
108/108 ━━━━━━━━━━━━━━━━━━━━ 22s 207ms/step - accuracy: 0.5266 - loss: 1.1513 - val_accuracy: 0.5150 - val_loss: 1.1641
Epoch 5/45
108/108 ━━━━━━━━━━━━━━━━━━━━ 22s 206ms/step - accuracy: 0.5406 - loss: 1.1059 - val_accuracy: 0.5637 - val_loss: 1.0470
Epoch 6/45
108/108 ━━━━━━━━━━━━━━━━━━━━ 22s 208ms/step - accuracy: 0.5864 - loss: 1.0744 - val_accuracy: 0.6204 - val_loss: 0.9577
Epoch 7/45
108/108 ━━━━━━━━━━━━━━━━━━━━ 22s 206ms/step - accuracy: 0.6278 - loss: 0.9700 - val_accuracy: 0.6644 - val_loss: 0.8815
Epoch 8/45
108/108 ━━━━━━━━━━━━━━━━━━━━ 41s 206ms/step - accuracy: 0.6431 - loss: 0

In [12]:
CNN_model.save("cnn_model.keras")